In [1]:
import pandas as pd
import numpy as np

# Ruta relativa al archivo CSV dentro de la carpeta 'data'
df_clientes = pd.read_csv('data/clients_202510280926.csv')

df_clientes.head()

,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,citizenship,document_type,document_number,registration_date,last_seen
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,Uruguay,CI,2719583-8,2025-05-02 00:00:00.000,2025-06-19 00:00:00.000
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,Uruguay,CI,1445199-8,2023-02-24 00:00:00.000,2023-12-09 00:00:00.000
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,Uruguay,CI,8090293-5,2024-10-03 00:00:00.000,2025-05-03 00:00:00.000
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,Uruguay,CI,7022674-5,2025-01-13 00:00:00.000,2025-08-11 00:00:00.000
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,Uruguay,CI,5918715-9,2025-03-09 00:00:00.000,2025-03-21 00:00:00.000


In [2]:
df_orders = pd.read_csv('data/orders_202510280929.csv')
df_orders.head()

,order_id,client_id,product_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost
0,1,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00333,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51
1,2,0c8380cb-a64e-4090-8b23-d1ddf32630e0,P00929,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76
2,3,bf486527-8182-490e-9e10-c79232c6b0c9,P00768,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42
3,5,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00610,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05
4,6,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,P00299,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08


In [3]:
df_products = pd.read_csv('data/products_202510280931.csv')
df_products.head()

,product_id,product_name,category,brand,last_updated,created_at
0,P00001,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31.000,2025-04-06 21:21:31.000
1,P00002,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2025-05-02 21:21:31.000
2,P00003,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2024-10-02 21:21:31.000
3,P00004,Lipton Ice Tea,Beverages,Unilever,2025-09-06 21:21:31.000,2025-04-21 21:21:31.000
4,P00005,Pepsi Max,Beverages,PepsiCo,2025-09-06 21:21:31.000,2025-04-22 21:21:31.000


In [4]:
# Combinar df_orders, df_clientes y df_products
# Buscamos automáticamente claves comunes (client / product) y realizamos LEFT JOINs
def find_common_key(a, b, candidates):
    for c in candidates:
        if c in a.columns and c in b.columns:
            return c
    common = set(a.columns).intersection(b.columns)
    if common:
        # retornamos la primera coincidencia detectada
        return list(common)[0]
    return None

client_candidates = ['client_id','clientId','cliente_id','clienteId','id_cliente','idCliente','customer_id','customerId']
product_candidates = ['product_id','productId','producto_id','productoId','id_producto','idProducto']

# Detectar clave para unir orders con clientes
key_client = find_common_key(df_orders, df_clientes, client_candidates)
print('client key used:', key_client)
if key_client is None:
    raise ValueError('No common client key found between orders and clients')

df_merged = df_orders.merge(df_clientes, how='left', left_on=key_client, right_on=key_client, suffixes=('_order','_client'))

# Detectar clave para unir con productos
key_product = find_common_key(df_merged, df_products, product_candidates)
print('product key used:', key_product)
if key_product is None:
    raise ValueError('No common product key found between merged orders+clients and products')

df_merged = df_merged.merge(df_products, how='left', left_on=key_product, right_on=key_product, suffixes=('','_product'))

# Resultado final
df_merged.head()

client key used: client_id
product key used: product_id


,order_id,client_id,product_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost,...,citizenship,document_type,document_number,registration_date,last_seen,product_name,category,brand,last_updated,created_at
0,1,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00333,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51,...,Uruguay,CI,4757489-7,2023-06-19 00:00:00.000,2024-09-12 00:00:00.000,Snickers,Snacks,Mars,2025-09-06 21:21:31.000,2025-06-13 21:21:31.000
1,2,0c8380cb-a64e-4090-8b23-d1ddf32630e0,P00929,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76,...,Uruguay,CI,8482755-0,2023-05-23 00:00:00.000,2023-09-26 00:00:00.000,Colgate Toothpaste,Personal Care,Colgate-Palmolive,2025-09-06 21:21:31.000,2025-02-25 21:21:31.000
2,3,bf486527-8182-490e-9e10-c79232c6b0c9,P00768,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42,...,Uruguay,CI,4458794-2,2023-01-15 00:00:00.000,2024-08-28 00:00:00.000,Gillette Razor,Personal Care,P&G,2025-09-06 21:21:31.000,2025-07-07 21:21:31.000
3,5,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00610,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05,...,Uruguay,CI,4757489-7,2023-06-19 00:00:00.000,2024-09-12 00:00:00.000,Fairy Dish Soap,Household,P&G,2025-09-06 21:21:31.000,2025-04-23 21:21:31.000
4,6,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,P00299,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08,...,Uruguay,CI,7026992-3,2024-04-19 00:00:00.000,2025-01-07 00:00:00.000,Snickers,Snacks,Mars,2025-09-06 21:21:31.000,2025-02-27 21:21:31.000


In [5]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82245 entries, 0 to 82244
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           82245 non-null  int64  
 1   client_id          82245 non-null  object 
 2   product_id         82245 non-null  object 
 3   product_category   82245 non-null  object 
 4   order_price        82245 non-null  float64
 5   order_timestamp    82245 non-null  object 
 6   order_status       82245 non-null  object 
 7   payment_method     82245 non-null  object 
 8   shipping_method    82245 non-null  object 
 9   shipping_cost      82245 non-null  float64
 10  first_name         82245 non-null  object 
 11  last_name          82245 non-null  object 
 12  gender             82245 non-null  object 
 13  age                82245 non-null  int64  
 14  email              82245 non-null  object 
 15  city               82245 non-null  object 
 16  address            822

In [6]:
df_merged = df_merged.drop(['order_id', 'product_id', 'category', 'email', 'document_type', 'document_number', 'last_name', 'first_name'], axis=1)
df_merged.head()

,client_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost,gender,age,...,address,postal_code,country,citizenship,registration_date,last_seen,product_name,brand,last_updated,created_at
0,6c5c87f2-49e9-4380-872d-71d9e3df63d6,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51,F,59,...,Av. Italia 5807,94536,Uruguay,Uruguay,2023-06-19 00:00:00.000,2024-09-12 00:00:00.000,Snickers,Mars,2025-09-06 21:21:31.000,2025-06-13 21:21:31.000
1,0c8380cb-a64e-4090-8b23-d1ddf32630e0,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76,F,46,...,Camino Maldonado 4174,46258,Uruguay,Uruguay,2023-05-23 00:00:00.000,2023-09-26 00:00:00.000,Colgate Toothpaste,Colgate-Palmolive,2025-09-06 21:21:31.000,2025-02-25 21:21:31.000
2,bf486527-8182-490e-9e10-c79232c6b0c9,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42,M,61,...,Gral. Flores 7898,26632,Uruguay,Uruguay,2023-01-15 00:00:00.000,2024-08-28 00:00:00.000,Gillette Razor,P&G,2025-09-06 21:21:31.000,2025-07-07 21:21:31.000
3,6c5c87f2-49e9-4380-872d-71d9e3df63d6,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05,F,59,...,Av. Italia 5807,94536,Uruguay,Uruguay,2023-06-19 00:00:00.000,2024-09-12 00:00:00.000,Fairy Dish Soap,P&G,2025-09-06 21:21:31.000,2025-04-23 21:21:31.000
4,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08,M,29,...,Av. Italia 7210,10761,Uruguay,Uruguay,2024-04-19 00:00:00.000,2025-01-07 00:00:00.000,Snickers,Mars,2025-09-06 21:21:31.000,2025-02-27 21:21:31.000


In [7]:
df_orders_products = df_orders.merge(df_products, on='product_id', how='left', suffixes=('_order','_product'))
df_orders_products.head()

,order_id,client_id,product_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost,product_name,category,brand,last_updated,created_at
0,1,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00333,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51,Snickers,Snacks,Mars,2025-09-06 21:21:31.000,2025-06-13 21:21:31.000
1,2,0c8380cb-a64e-4090-8b23-d1ddf32630e0,P00929,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76,Colgate Toothpaste,Personal Care,Colgate-Palmolive,2025-09-06 21:21:31.000,2025-02-25 21:21:31.000
2,3,bf486527-8182-490e-9e10-c79232c6b0c9,P00768,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42,Gillette Razor,Personal Care,P&G,2025-09-06 21:21:31.000,2025-07-07 21:21:31.000
3,5,6c5c87f2-49e9-4380-872d-71d9e3df63d6,P00610,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05,Fairy Dish Soap,Household,P&G,2025-09-06 21:21:31.000,2025-04-23 21:21:31.000
4,6,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,P00299,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08,Snickers,Snacks,Mars,2025-09-06 21:21:31.000,2025-02-27 21:21:31.000


In [8]:
df_orders_products = df_orders_products.drop(['order_id', 'product_id', 'category'], axis=1)
df_orders_products.head()

,client_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost,product_name,brand,last_updated,created_at
0,6c5c87f2-49e9-4380-872d-71d9e3df63d6,Snacks,29.51,2023-01-01 23:51:55.000,delivered,bank_transfer,standard,3.51,Snickers,Mars,2025-09-06 21:21:31.000,2025-06-13 21:21:31.000
1,0c8380cb-a64e-4090-8b23-d1ddf32630e0,Personal Care,23.40,2023-01-01 11:48:50.000,delivered,credit_card,standard,3.76,Colgate Toothpaste,Colgate-Palmolive,2025-09-06 21:21:31.000,2025-02-25 21:21:31.000
2,bf486527-8182-490e-9e10-c79232c6b0c9,Personal Care,94.41,2023-01-01 11:06:14.000,delivered,debit_card,express,10.42,Gillette Razor,P&G,2025-09-06 21:21:31.000,2025-07-07 21:21:31.000
3,6c5c87f2-49e9-4380-872d-71d9e3df63d6,Household,29.16,2023-01-01 20:55:35.000,delivered,credit_card,standard,6.05,Fairy Dish Soap,P&G,2025-09-06 21:21:31.000,2025-04-23 21:21:31.000
4,7ac7a850-4f19-4c85-921b-b0e5ba921ee9,Snacks,76.22,2023-01-01 02:49:09.000,delivered,debit_card,express,14.08,Snickers,Mars,2025-09-06 21:21:31.000,2025-02-27 21:21:31.000


In [9]:
df_orders_products['order_status'].value_counts()


order_status
delivered    81803
failed         442
Name: count, dtype: int64

In [10]:
df_orders_products.columns

Index(['client_id', 'product_category', 'order_price', 'order_timestamp',
       'order_status', 'payment_method', 'shipping_method', 'shipping_cost',
       'product_name', 'brand', 'last_updated', 'created_at'],
      dtype='object')

In [11]:
from datetime import datetime

# Copia del dataset original
df = df_orders_products.copy()

# Asegurar formato de fechas
df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])
fecha_ref = df['order_timestamp'].max()

# Calcular la categoría favorita del cliente
fav_category = (
    df.groupby(['client_id', 'product_category'])
    .size()
    .reset_index(name='count')
    .sort_values(['client_id', 'count'], ascending=[True, False])
    .drop_duplicates('client_id')
    .rename(columns={'product_category': 'favorite_category'})
)

# Agrupación principal por cliente
agg = df.groupby('client_id').agg(
    # Actividad general
    total_orders=('order_timestamp', 'count'),
    first_order_date=('order_timestamp', 'min'),
    last_order_date=('order_timestamp', 'max'),
    
    # Métricas monetarias
    total_spent=('order_price', 'sum'),
    average_order_value=('order_price', 'mean'),
    std_order_value=('order_price', 'std'),
    max_order_value=('order_price', 'max'),
    min_order_value=('order_price', 'min'),
    
    # Costos de envío
    total_shipping_cost=('shipping_cost', 'sum'),
    avg_shipping_cost=('shipping_cost', 'mean'),
    shipping_cost_ratio=('shipping_cost', lambda x: (
        x.sum() / df.loc[x.index, 'order_price'].sum()
        if df.loc[x.index, 'order_price'].sum() > 0 else 0
    )),
    
    # Diversidad
    unique_categories=('product_category', 'nunique'),
    unique_brands=('brand', 'nunique'),
    unique_payment_methods=('payment_method', 'nunique'),
    unique_shipping_methods=('shipping_method', 'nunique'),
    
    # Calidad / consistencia
    cancelled_orders=('order_status', lambda x: (x == 'failed').sum()
                      if 'failed' in x.unique() else 0),
    express_shipping_ratio=('shipping_method', lambda x: (
        x.str.contains('express', case=False)).mean()
        if x.notna().any() else 0),
    
    # Preferencias
    most_used_payment_method=('payment_method', lambda x: x.mode().iloc[0]
                              if not x.mode().empty else None),
    most_used_shipping_method=('shipping_method', lambda x: x.mode().iloc[0]
                               if not x.mode().empty else None)
).reset_index()

# --- Features temporales ---
agg['days_since_last_order'] = (fecha_ref - agg['last_order_date']).dt.days
agg['customer_lifespan_days'] = (fecha_ref - agg['first_order_date']).dt.days
agg['avg_days_between_orders'] = agg['customer_lifespan_days'] / agg['total_orders']

# --- Nueva feature propuesta ---
# Diferencia entre lo que suele tardar entre órdenes y lo que lleva sin comprar
agg['delay_vs_avg_interval'] = agg['days_since_last_order'] - agg['avg_days_between_orders']

# --- Meses activos ---
df['month'] = df['order_timestamp'].dt.to_period('M')
months_active = df.groupby('client_id')['month'].nunique().reset_index(name='months_active')
agg = agg.merge(months_active, on='client_id', how='left')

# --- Categoría favorita ---
agg = agg.merge(fav_category[['client_id', 'favorite_category']], on='client_id', how='left')

# Porcentaje del gasto en la categoría favorita
spend_by_cat = (
    df.groupby(['client_id', 'product_category'])['order_price']
    .sum()
    .reset_index()
)
spend_fav = spend_by_cat.merge(fav_category[['client_id', 'favorite_category']], on='client_id')
spend_fav = spend_fav[spend_fav['product_category'] == spend_fav['favorite_category']]
spend_fav = spend_fav.rename(columns={'order_price': 'favorite_category_spend'})
agg = agg.merge(spend_fav[['client_id', 'favorite_category_spend']], on='client_id', how='left')
agg['fav_category_spend_ratio'] = agg['favorite_category_spend'] / agg['total_spent']

# Reemplazar NaN por 0 en columnas numéricas
agg.fillna({
    'std_order_value': 0,
    'favorite_category_spend': 0,
    'fav_category_spend_ratio': 0,
    'months_active': 0,
}, inplace=True)

# --- Orden final de columnas ---
agg = agg[[
    'client_id',
    'total_orders',
    'first_order_date', 'last_order_date',
    'days_since_last_order', 'customer_lifespan_days', 'avg_days_between_orders', 'delay_vs_avg_interval',
    'total_spent', 'average_order_value', 'std_order_value', 'max_order_value', 'min_order_value',
    'total_shipping_cost', 'avg_shipping_cost', 'shipping_cost_ratio',
    'unique_categories', 'unique_brands', 'unique_payment_methods', 'unique_shipping_methods',
    'cancelled_orders', 'express_shipping_ratio',
    'months_active',
    'most_used_payment_method', 'most_used_shipping_method',
    'favorite_category', 'favorite_category_spend', 'fav_category_spend_ratio'
]]

agg.head()


,client_id,total_orders,first_order_date,last_order_date,days_since_last_order,customer_lifespan_days,avg_days_between_orders,delay_vs_avg_interval,total_spent,average_order_value,...,unique_payment_methods,unique_shipping_methods,cancelled_orders,express_shipping_ratio,months_active,most_used_payment_method,most_used_shipping_method,favorite_category,favorite_category_spend,fav_category_spend_ratio
0,00094d4c-1acd-46ec-a6d2-c35710c31005,2,2023-11-17 03:46:17,2024-04-27 17:58:43,248,410,205.000000,43.000000,133.85,66.925000,...,2,1,0,0.000000,2,credit_card,standard,Snacks,55.16,0.412103
1,0016d161-d6a2-42f3-bfd6-66aee6d163e3,4,2023-01-03 14:33:10,2023-07-11 15:17:43,539,728,182.000000,357.000000,235.11,58.777500,...,3,2,1,0.250000,4,debit_card,standard,Beverages,76.90,0.327081
2,0042e01f-9154-45fd-9c1d-499b3eb3f1c4,77,2023-01-16 05:43:41,2024-12-28 16:17:46,3,715,9.285714,-6.285714,3943.76,51.217662,...,5,3,1,0.207792,24,credit_card,standard,Household,1260.38,0.319588
3,0052b716-46cf-4540-aa23-409642dc4048,67,2023-01-12 04:44:03,2024-12-28 12:43:15,3,719,10.731343,-7.731343,3590.10,53.583582,...,5,3,0,0.179104,22,credit_card,standard,Snacks,876.60,0.244171
4,00679c7c-bd80-4a70-9052-830bc5b63ecf,79,2023-01-30 09:58:29,2024-12-17 22:38:56,14,701,8.873418,5.126582,4489.99,56.835316,...,5,3,0,0.177215,22,credit_card,standard,Technology,2046.11,0.455705


In [12]:
agg['cancelled_orders'].unique()

array([0, 1, 2])

In [13]:
df_analitico = df_clientes.merge(agg, on='client_id', how='left')
df_analitico.head()


,client_id,first_name,last_name,gender,age,email,city,address,postal_code,country,...,unique_payment_methods,unique_shipping_methods,cancelled_orders,express_shipping_ratio,months_active,most_used_payment_method,most_used_shipping_method,favorite_category,favorite_category_spend,fav_category_spend_ratio
0,21e5c13d-1c9a-4d00-9164-b72302d5edef,Tristán,Rojas,M,42,tristán.rojas13@example.com,Rivera,Bulevar Artigas 3757,28289,Uruguay,...,5.0,3.0,0.0,0.159091,22.0,debit_card,standard,Household,725.93,0.314776
1,36e48bdd-db11-4abe-9526-cfc90e68924d,Óscar,Barranco,M,58,óscar.barranco13@example.com,Salto,Bulevar Artigas 3911,76237,Uruguay,...,5.0,3.0,0.0,0.204918,24.0,credit_card,standard,Personal Care,2162.85,0.298863
2,145c22df-3579-412e-bc12-b4fce70abaf3,Macario,Querol,M,28,macario.querol67@example.com,Rivera,Av. Rivera 206,30926,Uruguay,...,5.0,3.0,0.0,0.135135,20.0,credit_card,standard,Technology,830.66,0.400013
3,90c4a925-e51f-4dac-9193-2d9aec97a472,Ramiro,Sanchez,M,19,ramiro.sanchez53@example.com,Salto,Av. 18 de Julio 6324,22676,Uruguay,...,5.0,3.0,0.0,0.230769,9.0,credit_card,standard,Technology,319.26,0.519637
4,853f711a-4c36-40b4-b5d0-6207152cd793,Juan Francisco,Flor,M,36,juan francisco.flor68@example.com,Mercedes,Av. 18 de Julio 6301,20328,Uruguay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_analitico['total_orders'].isna().sum()

np.int64(102)

In [15]:
# Crear df_estudio como copia filtrada de df_orders_products
df_estudio = df_orders_products.copy()
# Asegurar que 'order_timestamp' es datetime y filtrar al periodo solicitado (inclusive)
df_estudio['order_timestamp'] = pd.to_datetime(df_estudio['order_timestamp'])
start = pd.to_datetime('2024-01-01')
end = pd.to_datetime('2024-06-01')
df_estudio = df_estudio[(df_estudio['order_timestamp'] >= start) & (df_estudio['order_timestamp'] <= end)].reset_index(drop=True)
print('Filas en df_estudio tras filtrar:', df_estudio.shape[0])
df_estudio.head()

Filas en df_estudio tras filtrar: 16895


,client_id,product_category,order_price,order_timestamp,order_status,payment_method,shipping_method,shipping_cost,product_name,brand,last_updated,created_at
0,efbc66db-49df-46df-ae21-19247fc57715,Snacks,1.56,2024-01-17 19:55:31,delivered,cash,standard,3.35,Doritos Nacho,Frito-Lay,2025-09-06 21:21:31.000,2025-07-14 21:21:31.000
1,05d81518-ee99-487b-9fda-ee158ada7fa7,Snacks,63.91,2024-01-28 07:55:28,delivered,cash,standard,6.18,Snickers,Mars,2025-09-06 21:21:31.000,2024-09-23 21:21:31.000
2,be1c294f-5977-47b4-aa95-3e3b83bc71b9,Snacks,79.98,2024-01-28 03:36:11,delivered,cash,standard,6.02,KitKat Bar,Nestlé,2025-09-06 21:21:31.000,2024-09-13 21:21:31.000
3,0f188201-bb53-4a98-b091-5503e9d5240d,Snacks,64.06,2024-02-05 01:19:23,delivered,cash,express,11.84,Oreo Cookies,Mondelez,2025-09-06 21:21:31.000,2024-11-24 21:21:31.000
4,49305ea0-41fa-4c75-8d80-6216c476a3a6,Snacks,29.12,2024-02-11 02:38:38,delivered,cash,standard,3.24,Doritos Nacho,Frito-Lay,2025-09-06 21:21:31.000,2024-12-19 21:21:31.000


In [ ]:
# --- Cálculo de métricas sobre df_estudio (fecha de referencia 2024-06-01) ---
from datetime import datetime

# Usar df_estudio (ya filtrado al periodo)
df = df_estudio.copy()

# Asegurar formato de fechas
df['order_timestamp'] = pd.to_datetime(df['order_timestamp'])
# Fecha de referencia fija para el análisis
fecha_ref = pd.to_datetime('2024-06-01')
print('Fecha de referencia usada:', fecha_ref)

# Calcular la categoría favorita del cliente
fav_category = (
    df.groupby(['client_id', 'product_category'])
    .size()
    .reset_index(name='count')
    .sort_values(['client_id', 'count'], ascending=[True, False])
    .drop_duplicates('client_id')
    .rename(columns={'product_category': 'favorite_category'})
)

# Agrupación principal por cliente (métricas principales)
agg = df.groupby('client_id').agg(
    total_orders=('order_timestamp', 'count'),
    first_order_date=('order_timestamp', 'min'),
    last_order_date=('order_timestamp', 'max'),
    total_spent=('order_price', 'sum'),
    average_order_value=('order_price', 'mean'),
    std_order_value=('order_price', 'std'),
    max_order_value=('order_price', 'max'),
    min_order_value=('order_price', 'min'),
    total_shipping_cost=('shipping_cost', 'sum'),
    avg_shipping_cost=('shipping_cost', 'mean'),
    shipping_cost_ratio=('shipping_cost', lambda x: (
        x.sum() / df.loc[x.index, 'order_price'].sum()
        if df.loc[x.index, 'order_price'].sum() > 0 else 0
    )),
    unique_categories=('product_category', 'nunique'),
    unique_brands=('brand', 'nunique'),
    unique_payment_methods=('payment_method', 'nunique'),
    unique_shipping_methods=('shipping_method', 'nunique'),
    cancelled_orders=('order_status', lambda x: (x == 'failed').sum() if 'failed' in x.unique() else 0),
    express_shipping_ratio=('shipping_method', lambda x: (x.str.contains('express', case=False)).mean() if x.notna().any() else 0),
    most_used_payment_method=('payment_method', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    most_used_shipping_method=('shipping_method', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()

# --- Features temporales (usando fecha_ref) ---
agg['days_since_last_order'] = (fecha_ref - agg['last_order_date']).dt.days
agg['customer_lifespan_days'] = (fecha_ref - agg['first_order_date']).dt.days
agg['avg_days_between_orders'] = agg['customer_lifespan_days'] / agg['total_orders']
agg['delay_vs_avg_interval'] = agg['days_since_last_order'] - agg['avg_days_between_orders']

# Meses activos
df['month'] = df['order_timestamp'].dt.to_period('M')
months_active = df.groupby('client_id')['month'].nunique().reset_index(name='months_active')
agg = agg.merge(months_active, on='client_id', how='left')

# Categoria favorita y gasto en la favorita
agg = agg.merge(fav_category[['client_id', 'favorite_category']], on='client_id', how='left')
spend_by_cat = df.groupby(['client_id', 'product_category'])['order_price'].sum().reset_index()
spend_fav = spend_by_cat.merge(fav_category[['client_id', 'favorite_category']], on='client_id')
spend_fav = spend_fav[spend_fav['product_category'] == spend_fav['favorite_category']]
spend_fav = spend_fav.rename(columns={'order_price': 'favorite_category_spend'})
agg = agg.merge(spend_fav[['client_id', 'favorite_category_spend']], on='client_id', how='left')
agg['fav_category_spend_ratio'] = agg['favorite_category_spend'] / agg['total_spent']

# Reemplazar NaN por 0 en columnas numéricas relevantes
agg.fillna({
    'std_order_value': 0,
    'favorite_category_spend': 0,
    'fav_category_spend_ratio': 0,
    'months_active': 0,
}, inplace=True)

# Orden final de columnas (opcional)
cols = [
    'client_id', 'total_orders', 'first_order_date', 'last_order_date',
    'days_since_last_order', 'customer_lifespan_days', 'avg_days_between_orders', 'delay_vs_avg_interval',
    'total_spent', 'average_order_value', 'std_order_value', 'max_order_value', 'min_order_value',
    'total_shipping_cost', 'avg_shipping_cost', 'shipping_cost_ratio',
    'unique_categories', 'unique_brands', 'unique_payment_methods', 'unique_shipping_methods',
    'cancelled_orders', 'express_shipping_ratio', 'months_active',
    'most_used_payment_method', 'most_used_shipping_method',
    'favorite_category', 'favorite_category_spend', 'fav_category_spend_ratio'
]
agg = agg[[c for c in cols if c in agg.columns]]

# Mostrar resultado y crear df_analitico
print('Clientes en agg:', agg.shape[0])
display(agg.head())

df_analitico = df_clientes.merge(agg, on='client_id', how='left')
print('Filas en df_analitico:', df_analitico.shape[0])
display(df_analitico.head())